In [1]:
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import collections

**Load the data and split them**

In [8]:
oush_df = pd.read_csv('./reuters_10class.csv')
print(len(oush_df))
# reuters_df = pd.read_csv('./reuters1000.csv')
# reuters_df.to_csv('./reuters1000.csv',index=False)
# print('Finish')
train_size = int(0.7 * len(oush_df))
oush_Y_train = oush_df.iloc[:train_size,1:].values
oush_Y_test = oush_df.iloc[train_size:,1:].values
# reuters_Y_train = reuters_df.iloc[:train_size,1:5].values
# reuters_Y_test = reuters_df.iloc[train_size:,1:5].values
# print(reuters_Y_train.shape)

16876


In [9]:
def cleanse(text):
    text = text.lower()
    filters = ['"','#','$','%','&','\(','\)','\*','\+',',','-','\.','/',':',';','<','=','>','\?','@','\[','\\','\]'
               ,'^','_',' ','\{','\|','\}','~','\t','\n','\x97','\x96']
    text = re.sub("<.*?>",' ',text)
    text = re.sub("|".join(filters)," ",text)
    return text

In [10]:
def encoder(dataset,max_feature):
    content = dataset.iloc[:,0].values.tolist()
    freq=pd.Series(' '.join(content).split()).value_counts()
    total_size = len(content)
    new_contents = []
    document = []
    from nltk.stem import PorterStemmer
    st = PorterStemmer()
    for text in content:
        new_text = cleanse(text)
        split_content = word_tokenize(new_text)
        words = [word for word in split_content if word not in stopwords.words('english')]
        words = [WordNetLemmatizer().lemmatize(w) for w in words]
        # words = [st.stem(w) for w in words]
        words = [w for w in words if w not in freq[:10]]
        
        new_contents.append([(' '.join(words))])

    # counter = collections.Counter([tk for st in new_contents for tk in st[0].split()])
    # counter = list(dict(filter(lambda x: x[1] >= 1, counter.items())).keys())
    # contents = []

    # for text in new_contents:
    #     words = [w for w in text[0].split() if w in counter]
    #     if len(words):
    #         contents.append([' '.join(words)])
    # new_contents = contents

    # features = len(counter)
    documents = [new_content[0] for new_content in new_contents]
    vectorize = CountVectorizer(max_features=400)
    vectorize.fit(documents)
    features = len(vectorize.vocabulary_)
    X_train = np.ones(shape=[train_size,features],dtype=np.int64)
    X_test = np.ones(shape=[len(new_contents)-train_size,features])
    for idx,new_content in enumerate(new_contents):
        word_vector = vectorize.transform(new_content).toarray()
        if idx >= train_size: 
            X_test[idx-train_size] = word_vector
        else:
             X_train[idx] = word_vector
    return X_train,X_test

max_feature = 400
oush_X_train,oush_X_test = encoder(oush_df,max_feature)
print(oush_X_train.shape,oush_Y_test.shape)
# reuters_X_train,reuters_X_test = encoder(reuters_df,max_feature)

(11813, 400) (5063, 9)


In [11]:
oush_X_test.shape

(5063, 400)

In [12]:
np.save('./traindataReuters_10class.npy',oush_X_train)
np.save('./trainlabelReuters_10class.npy',oush_Y_train)                               
np.save('./testdataReuters_10class.npy',oush_X_test)
np.save('./testlabelReuters_10class.npy',oush_Y_test)
print('Finish')

Finish
